In [1]:
!python3 --version

Python 3.13.2


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc_greentaxi_tripdata_experiment')

<Experiment: artifact_location='/Users/mac/Projects/MLops_datatalks/mlruns/1', creation_time=1748148094528, experiment_id='1', last_update_time=1748148094528, lifecycle_stage='active', name='nyc_greentaxi_tripdata_experiment', tags={}>

Preprocessing

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime)
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df = df[((df['duration'] > 1) & (df['duration'] <= 60))]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('green_tripdata/green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata/green_tripdata_2021-02.parquet')

In [6]:
df_train.shape, df_val.shape

((73833, 21), (61889, 21))

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_'  +  df_val['DOLocationID']

In [8]:
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_DO
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,3.933333,43_151
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,8.750000,166_239
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,5.966667,41_42
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,7.083333,168_75
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,...,0.96,0.0,None,0.3,5.76,1.0,1.0,0.00,2.316667,75_75


In [9]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [10]:
X_train, y_train
X_val, y_val

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 118522 stored elements and shape (61889, 13221)>,
 array([17.91666667,  6.5       , 15.25      , ..., 14.        ,
         7.        ,  7.        ], shape=(61889,)))

In [11]:
from math import sqrt

Train with Linear Regression

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse_lr = sqrt(mean_squared_error(y_val, y_pred))
rmse_lr

7.756932266972238

In [13]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Train with Lasso and log with MLflow

In [14]:
with mlflow.start_run():
    mlflow.set_tag("engineer", "richkinwe")

    mlflow.log_param("train_data_path", "green_tripdata/green_tripdata_2021-01.parquet")
    mlflow.log_param("val_data_path", "green_tripdata/green_tripdata_2021-02.parquet")

    alpha = 0.00001
    mlflow.log_param("alpha", alpha)
    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)

    rmse_ls = sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse_ls)

Train and tune hyperparameters with xgboost and log with MLflow

In [15]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
val = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("engineer", "richkinwe")
        mlflow.log_param("train_data_path", "green_tripdata/green_tripdata_2021-01.parquet")
        mlflow.log_param("val_data_path", "green_tripdata/green_tripdata_2021-02.parquet")
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(val, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(val)
        rmse = sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [22]:
mlflow.xgboost.autolog(disable=False)

2025/05/31 17:04:54 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 3.0.0, but the installed version is 3.0.1. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.


In [21]:
import mlflow.xgboost


with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    val = xgb.DMatrix(X_val, label=y_val)

    best_params = {
    'max_depth': 30,
    'learning_rate': 0.0959,
    'reg_alpha': 0.0181,
    'reg_lambda': 0.0117,
    'min_child_weight': 1.0606,
    'objective': 'reg:linear',
    'seed': 42
}
    
    mlflow.log_params(best_params)

    booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=50,
            evals=[(val, 'validation')],
            early_stopping_rounds=50
        )
    
    y_pred = booster.predict(val)
    rmse = sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    # with open('models.preprocessor.b', 'wb') as f_out:
    #     pickle.dump(dv, f_out)

    # mlflow.log_artifact('models.preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='models_xgboost_mlflow')

/Users/mac/Projects/MLops_datatalks/mlflow-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [17:01:05] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44184
[1]	validation-rmse:10.76905
[2]	validation-rmse:10.18099
[3]	validation-rmse:9.67144
[4]	validation-rmse:9.22906
[5]	validation-rmse:8.84628
[6]	validation-rmse:8.51703
[7]	validation-rmse:8.23573
[8]	validation-rmse:7.99501
[9]	validation-rmse:7.78775
[10]	validation-rmse:7.61110
[11]	validation-rmse:7.45933
[12]	validation-rmse:7.33090
[13]	validation-rmse:7.22033
[14]	validation-rmse:7.12747
[15]	validation-rmse:7.04858
[16]	validation-rmse:6.98116
[17]	validation-rmse:6.92249
[18]	validation-rmse:6.87272
[19]	validation-rmse:6.82975
[20]	validation-rmse:6.79302
[21]	validation-rmse:6.76016
[22]	validation-rmse:6.73249
[23]	validation-rmse:6.70755
[24]	validation-rmse:6.68618
[25]	validation-rmse:6.66658
[26]	validation-rmse:6.64923
[27]	validation-rmse:6.63447
[28]	validation-rmse:6.62169
[29]	validation-rmse:6.60994
[30]	validation-rmse:6.60004
[31]	validation-rmse:6.59057
[32]	validation-rmse:6.58291
[33]	validation-rmse:6.57608
[34]	validation-rmse:

/Users/mac/Projects/MLops_datatalks/mlflow-env/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [17:01:41] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/31 17:01:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
